# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - CP modeling of wreath_N/N</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Code modified from: https://www.kaggle.com/code/siukeitin/santa-2023-cp-modeling-of-wreath-n-n-for-n-6-7

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
from scipy.sparse import identity, csr_matrix
import math
from ortools.sat.python import cp_model
import time, datetime
from ast import literal_eval 
import numpy as np
import pandas as pd
from tqdm import tqdm
import zipfile
import sqlite3

<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [2]:
with zipfile.ZipFile('../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col = 'id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [3]:
def build_automaton(allowed_moves,initial_state,solution_state,wildcards,sparse_matrix=False,progress=False):
    P = []
    for m in allowed_moves:
        x = allowed_moves[m]
        if sparse_matrix:
            P.append(csr_matrix(identity(len(x)))[x])
        else:
            P.append(np.eye(len(x),dtype='int')[x])
    
    if progress:
        N = (len(initial_state)+2)//2
        pbar = tqdm(total=math.factorial(len(initial_state))//math.factorial(N-2)**2//2)
    
    states = [list(initial_state)]
    if progress:
        pbar.update()
    final_states = []
    arcs = []
    i = 0
    while i < len(states):
        s0 = states[i]
        if np.sum(np.array(s0)!=solution_state)<=wildcards:
            arcs.append((i,0,i)) # self loop
            final_states.append(i)
        else:
            for j,p in enumerate(P):
                # forward
                s1 = list(p@np.array(s0))
                if s1 in states:
                    i1 = states.index(s1)
                else:
                    states.append(s1)
                    i1 = len(states)-1
                    if progress:
                        pbar.update()
                arcs.append((i,j+1,i1))
                # backwards
                s1 = list(p.T@np.array(s0))
                if s1 in states:
                    i1 = states.index(s1)
                else:
                    states.append(s1)
                    i1 = len(states)-1
                    if progress:
                        pbar.update()
                arcs.append((i,-j-1,i1))            
        i += 1
        
    if progress:
        pbar.close()
        
    return states, arcs, final_states

In [4]:
def solve_wreath_puzzle(puzzle, allowed_moves, verbose=False):

    t0 = time.time()
    initial_state = np.array(puzzle.initial_state.split(';'))
    initial_state = np.vectorize({'A':0,'B':1,'C':2}.get)(initial_state)
    solution_state = np.array(puzzle.solution_state.split(';'))
    solution_state = np.vectorize({'A':0,'B':1,'C':2}.get)(solution_state)
    wildcards = puzzle.num_wildcards

    if verbose:
        print('Building automaton...')
    states, arcs, final_states = build_automaton(allowed_moves,initial_state,solution_state,
                                                 wildcards=wildcards,sparse_matrix=False,progress=verbose)

    max_steps = 10
    done = False
    M = 100
    
    if verbose:
        print('Solving automaton...')
    while not done and max_steps <= M: # prevent infinite loop
        
        model = cp_model.CpModel()

        t = [model.NewIntVar(-2,2,F't_{i}') for i in range(max_steps)]
        z = [model.NewBoolVar(F'z_{i}') for i in range(max_steps)]

        starting_state = 0
        model.AddAutomaton(t, starting_state, final_states, arcs)
        for i in range(max_steps):
            model.Add(t[i] == 0).OnlyEnforceIf(z[i].Not())

        model.Minimize(cp_model.LinearExpr.Sum(z))

        solver = cp_model.CpSolver()
        status = solver.Solve(model)

        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            moves = np.array([solver.Value(t[i]) for i in range(max_steps)])
            moves = moves[moves!=0]
            solution = np.vectorize({-2:'-r',-1:'-l',1:'l',2:'r'}.get)(moves)
            solution = '.'.join(solution)
            done = True
            if verbose:
                print('Solution found: {0} in {1} moves. Elapsed time {2}'.format(
                    "Optimal" if status == cp_model.OPTIMAL else "Feasible", len(moves), 
                    str(datetime.timedelta(seconds=time.time()-t0))))
        else:
            max_steps += 5
            
    if done:
        return solution
    else:
        if verbose:
            print(F'No solution found in {M} moves. Elapsed time {str(datetime.timedelta(seconds=time.time()-t0))}')
        return None

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [6]:
puzzle_id = 334
puzzle = puzzles.iloc[puzzle_id]
allowed_moves = literal_eval(puzzle_info.loc[puzzle.puzzle_type].allowed_moves)
solution = solve_wreath_puzzle(puzzle, allowed_moves, verbose=True)
solution

Building automaton...


  0%|          | 103152/938303560162606353408 [20:07<7581303878664303:30:08, 34.38it/s] 
KeyboardInterrupt



In [ ]:
database_file = '../solutions.db'
solution_method = 'iterative replacement of K successive moves'


conn = sqlite3.connect(database_file)
cursor = conn.cursor()

for game_id in range(333, 398):# range(187, 281):
    # game_id = 1
    puzzle = puzzles.loc[game_id]
    time_limit = 72 * 60 * 60
    
    select_query = "SELECT moves, solution_method FROM solutions WHERE id = ?"
            
    # Execute the query
    cursor.execute(select_query, (game_id,))
    response = cursor.fetchone()
    
    sample_moves = response[0].split(".")
    best_solution_method = response[1]
    
    # sample_submission = pd.read_csv("data/sample_submission.csv").set_index("id").loc[game_id]
    # sample_moves = sample_submission["moves"].split(".")
    print(puzzle)
    print(f"Sample score: {len(sample_moves)}")
    
    moves = get_moves(puzzle["puzzle_type"])
    print(f"Number of moves: {len(moves)}")
    
    K = 2
    while True:
        try:
            shortest_path = get_shortest_path(moves, K, None if K == 2 else 10000000)
        except ExceedMaxSizeError:
            break
        K += 1
    print(f"K: {K}")
    print(f"Number of shortest_path: {len(shortest_path)}")
    
    current_state = puzzle["initial_state"].split(";")
    current_solution = list(sample_moves)
    initial_score = len(current_solution)
    start_time = time.time()
    
    with tqdm(total=len(current_solution) - K, desc=f"Score: {len(current_solution)} (-0)") as pbar:
        step = 0
        while step + K < len(current_solution) and time.time() - start_time < time_limit:
            replaced_moves = current_solution[step : step + K + 1]
            state_before = current_state
            state_after = current_state
            for move_name in replaced_moves:
                state_after = [state_after[i] for i in moves[move_name]]
    
            found_moves = None
            for perm, move_names in shortest_path.items():
                for i, j in enumerate(perm):
                    if state_after[i] != state_before[j]:
                        break
                else:
                    found_moves = move_names
                    break
    
            if found_moves is not None:
                length_before = len(current_solution)
                current_solution = current_solution[:step] + list(found_moves) + current_solution[step + K + 1 :]
                pbar.update(length_before - len(current_solution))
                pbar.set_description(f"Score: {len(current_solution)} ({len(current_solution) - initial_score})")
                for _ in range(K):
                    if step == 0:
                        break
                    step -= 1
                    pbar.update(-1)
                    move_name = current_solution[step]
                    move_name = move_name[1:] if move_name.startswith("-") else f"-{move_name}"
                    current_state = [current_state[i] for i in moves[move_name]]
            else:
                current_state = [current_state[i] for i in moves[current_solution[step]]]
                step += 1
                pbar.update(1)
    
    # validation
    state = puzzle["initial_state"].split(";")
    for move_name in current_solution:
        state = [state[i] for i in moves[move_name]]
    # print(puzzle["solution_state"])
    # print(state)
    assert puzzle["solution_state"].split(";") == state
    current_solution_length = len(current_solution)
    if current_solution_length < len(sample_moves):
        # print('.'.join(current_solution))
        print(f'Improvement to {game_id}')
        # Insert the moves into the database
        insert_query = "INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)"
        cursor.execute(insert_query, (game_id, '.'.join(current_solution), current_solution_length, f'{solution_method}: {best_solution_method}'))
        conn.commit()
# Commit the changes and close the connection
conn.commit()
conn.close()

puzzle_type                                            wreath_21/21
solution_state    C;A;A;A;A;A;C;A;A;A;A;A;A;A;A;A;A;A;A;A;A;B;B;...
initial_state     B;A;A;B;B;B;B;A;B;A;B;B;B;B;A;B;B;A;A;A;A;A;B;...
num_wildcards                                                     0
Name: 333, dtype: object
Sample score: 3290
Number of moves: 4
K: 15
Number of shortest_path: 7470398


Score: 3044 (-246):  51%|█████     | 1672/3275 [2:41:10<1:16:21,  2.86s/it]

In [ ]:
current_solution